# Predicting YCombinator Startup Performance

### Load dataset

In [48]:
import pandas as pd
import os
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [49]:
badges = pd.read_csv('/Users/siddarvind/Downloads/archive-7/badges.csv')
companies = pd.read_csv('/Users/siddarvind/Downloads/archive-7/companies.csv')
founders = pd.read_csv('/Users/siddarvind/Downloads/archive-7/founders.csv')
industries = pd.read_csv('/Users/siddarvind/Downloads/archive-7/industries.csv')
prior_companies = pd.read_csv('/Users/siddarvind/Downloads/archive-7/prior_companies.csv')
regions = pd.read_csv('/Users/siddarvind/Downloads/archive-7/regions.csv')
schools = pd.read_csv('/Users/siddarvind/Downloads/archive-7/schools.csv')
tags = pd.read_csv('/Users/siddarvind/Downloads/archive-7/tags.csv')

badges['badge_num'] = badges.groupby('id').cumcount() + 1
df_pivot = badges.pivot(index='id', columns='badge_num', values='badge')
df_pivot.columns = [f'badge_{int(col)}' for col in df_pivot.columns]
badges = df_pivot.reset_index()

industries['industry_num'] = industries.groupby('id').cumcount() + 1
df_pivot = industries.pivot(index='id', columns='industry_num', values='industry')
df_pivot.columns = [f'industry_{int(col)}' for col in df_pivot.columns]
industries = df_pivot.reset_index()

tags['tag_num'] = tags.groupby('id').cumcount() + 1
df_pivot = tags.pivot(index='id', columns='tag_num', values='tag')
df_pivot.columns = [f'tag_{int(col)}' for col in df_pivot.columns]
tags = df_pivot.reset_index()

schools = schools.drop_duplicates(subset='hnid', keep='first')
ivy_league_schools = [
    'Harvard University',
    'Yale University',
    'Princeton University',
    'Columbia University',
    'University of Pennsylvania',
    'Dartmouth College',
    'Brown University',
    'Cornell University'
]

#TODO why all false?
def is_ivy_league(school_name):
    # Strip whitespace and use fuzzy matching
    school_name = str(school_name)
    school_name = school_name.strip().lower()
    # Get the best match and its score
    best_match, score = process.extractOne(school_name, ivy_league_schools, scorer=fuzz.ratio)
    return score >= 80

schools['is_ivy_league'] = schools['school'].apply(is_ivy_league)
faang_companies = [
    'Facebook',
    'Apple',
    'Meta',
    'Amazon',
    'Netflix',
    'Microsoft',
    'Google'
]
def is_faang(company_name):
    company_name = str(company_name)
    company_name = company_name.strip().lower()
    best_match, score = process.extractOne(company_name, faang_companies, scorer=fuzz.ratio)
    return score >= 80  # Threshold for matching

prior_companies['is_faang'] = prior_companies['company'].apply(is_faang)

founder_dfs = [founders, schools, prior_companies]
company_dfs = [badges, companies, industries, regions, tags]

for i, df in enumerate(company_dfs):
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)

merged_df = company_dfs[0]

for i, df in enumerate(company_dfs[1:], start=1):
    merged_df = pd.merge(merged_df, df, on='id', how='inner')

merged_df.rename(columns={'slug': 'company_slug'}, inplace=True)
#merged_df = pd.merge(merged_df, founders, on='company_slug', how='inner')
#merged_df = pd.merge(merged_df, schools, on='hnid', how='inner')
#merged_df = pd.merge(merged_df, prior_companies, on='hnid', how='inner')

merged_df


,id,badge_1,badge_2,badge_3,badge_4,name,company_slug,website,smallLogoUrl,oneLiner,...,industry_2,region,country,address,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6
0,5,isHiring,NaN,NaN,NaN,CircuitHub,circuithub,https://circuithub.com,https://bookface-images.s3.amazonaws.com/small...,Electronics factory automation,...,Manufacturing and Robotics,Europe,United Kingdom,"London, UK",Hard Tech,Hardware,Robotics,NaN,NaN,NaN
1,8,topCompany,highlightWomen,NaN,NaN,PlanGrid,plangrid,http://plangrid.com,https://bookface-images.s3.amazonaws.com/small...,Mobile applications for the construction indus...,...,Construction,America / Canada,United States of America,"San Francisco, CA, USA",Construction,NaN,NaN,NaN,NaN,NaN
2,10,highlightWomen,NaN,NaN,NaN,The Muse,the-muse,http://themuse.com,https://bookface-images.s3.amazonaws.com/small...,The Muse is values-based job search & hiring f...,...,Recruiting and Talent,America / Canada,United States of America,"New York, NY, USA",Marketplace,SaaS,Recruiting,NaN,NaN,NaN
3,11,highlightBlack,NaN,NaN,NaN,SendHub,sendhub,http://sendhub.com,https://bookface-images.s3.amazonaws.com/small...,Simple Business SMS. SMBs to Large Enterprise.,...,Productivity,America / Canada,United States of America,"Pleasanton, CA, USA",Messaging,Enterprise,SMB,NaN,NaN,NaN
4,19,isHiring,NaN,NaN,NaN,Rescale,rescale,https://rescale.com,https://bookface-images.s3.amazonaws.com/small...,High Performance Computing Built for the Cloud,...,"Engineering, Product and Design",America / Canada,United States of America,"San Francisco, CA, USA",Cloud Computing,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,29964,isHiring,NaN,NaN,NaN,Overlap,overlap,https://www.joinoverlap.com/,https://bookface-images.s3.amazonaws.com/small...,Overlap is Perplexity for video & audio content,...,Content,America / Canada,United States of America,"San Francisco, CA, USA",Consumer,Media,Podcasts,AI,NaN,NaN
1979,29967,highlightBlack,NaN,NaN,NaN,Village Labs,village-labs,https://villagelabs.app/,https://bookface-images.s3.amazonaws.com/small...,Helping America's businesses become employee o...,...,Human Resources,America / Canada,United States of America,"New York, NY, USA",Fintech,Human Resources,NaN,NaN,NaN,NaN
1980,29972,highlightBlack,NaN,NaN,NaN,Affil.ai,affil-ai,https://affil.ai/,https://bookface-images.s3.amazonaws.com/small...,AI Affiliate Network For Financial Companies,...,Marketing,America / Canada,United States of America,"San Francisco, CA, USA",SaaS,B2B,Compliance,Marketing,AI,NaN
1981,29977,highlightWomen,NaN,NaN,NaN,Merlin AI,merlin-ai,https://www.merlinai.co,https://bookface-images.s3.amazonaws.com/small...,AI powered ERP for Construction,...,NaN,America / Canada,United States of America,"Los Angeles, CA, USA",SaaS,Construction,B2B,Enterprise Software,AI,NaN


### Removing the duplicates

In [50]:
def choose_row(group):
    na_counts = group.isna().sum(axis=1)
    return group.loc[na_counts.idxmin()]

merged_df = merged_df.groupby('name').apply(choose_row).reset_index(drop=True)
merged_df

/var/folders/cv/vszp_60n2yx3c1fttzx00nb40000gn/T/ipykernel_6146/2443777121.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby('name').apply(choose_row).reset_index(drop=True)


,id,badge_1,badge_2,badge_3,badge_4,name,company_slug,website,smallLogoUrl,oneLiner,...,industry_2,region,country,address,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6
0,29798,highlightBlack,NaN,NaN,NaN,&AI,ai-2,https://www.tryandai.com/,https://bookface-images.s3.amazonaws.com/small...,The secure patent due diligence and management...,...,Legal,America / Canada,United States of America,"San Francisco, CA, USA",SaaS,B2B,LegalTech,NaN,NaN,NaN
1,1700,highlightWomen,NaN,NaN,NaN,10 By 10,10-by-10,https://10by10.io,https://bookface-images.s3.amazonaws.com/small...,10by10 is a unique marketplace in the recruiti...,...,Recruiting and Talent,America / Canada,United States of America,"San Francisco, CA, USA",Artificial Intelligence,Marketplace,Recruiting,NaN,NaN,NaN
2,12286,highlightBlack,highlightWomen,NaN,NaN,1910 Genetics,1910-genetics,http://1910genetics.com,https://bookface-images.s3.amazonaws.com/small...,Make undruggable targets a thing of the past,...,Therapeutics,America / Canada,United States of America,"Cambridge, MA, USA",AI-powered Drug Discovery,NaN,NaN,NaN,NaN,NaN
3,27925,isHiring,NaN,NaN,NaN,222,222,https://222.place,https://bookface-images.s3.amazonaws.com/small...,marketplace facilitating IRL social experience...,...,NaN,America / Canada,United States of America,"New York, NY, USA",Machine Learning,Marketplace,Consumer,Social,AI,NaN
4,12252,highlightBlack,NaN,NaN,NaN,54Gene,54gene,http://www.54gene.com,https://bookface-images.s3.amazonaws.com/small...,Precision medicine for Africans and the global...,...,Consumer Health and Wellness,Africa,Nigeria,"Lagos, Nigeria",Genomics,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1960,25691,isHiring,NaN,NaN,NaN,voize,voize,https://voize.de/,https://bookface-images.s3.amazonaws.com/small...,App for healthcare workers to fill out forms u...,...,Productivity,Europe,Germany,"Berlin, Germany",Healthcare,NaN,NaN,NaN,NaN,NaN
1961,21906,highlightWomen,NaN,NaN,NaN,webapp.io,webapp-io,https://webapp.io,https://bookface-images.s3.amazonaws.com/small...,Build webapps faster with 10x better CI/CD + p...,...,"Engineering, Product and Design",America / Canada,Canada,"Toronto, ON, Canada",Developer Tools,SaaS,B2B,NaN,NaN,NaN
1962,22856,isHiring,NaN,NaN,NaN,weweb.io,weweb-io,https://www.weweb.io/,https://bookface-images.s3.amazonaws.com/small...,WeWeb is a no-code front-end builder that conn...,...,Operations,Europe,France,"Paris, France",Developer Tools,Web Development,NaN,NaN,NaN,NaN
1963,25977,highlightWomen,NaN,NaN,NaN,yhangry,yhangry,http://www.yhangry.com,https://bookface-images.s3.amazonaws.com/small...,Marketplace for private chefs,...,Food and Beverage,Europe,United Kingdom,"London, UK",Marketplace,NaN,NaN,NaN,NaN,NaN


### Create variables

In [51]:
def choose_row(group):
    na_counts = group.isna().sum(axis=1)
    return group.loc[na_counts.idxmin()]

merged_df = merged_df.groupby('name').apply(choose_row).reset_index(drop=True)

print("DataFrame after removing duplicates with the most NaNs:")
print(merged_df)

DataFrame after removing duplicates with the most NaNs:
         id          badge_1         badge_2 badge_3 badge_4           name  \
0     29798   highlightBlack             NaN     NaN     NaN            &AI   
1      1700   highlightWomen             NaN     NaN     NaN       10 By 10   
2     12286   highlightBlack  highlightWomen     NaN     NaN  1910 Genetics   
3     27925         isHiring             NaN     NaN     NaN            222   
4     12252   highlightBlack             NaN     NaN     NaN         54Gene   
...     ...              ...             ...     ...     ...            ...   
1960  25691         isHiring             NaN     NaN     NaN          voize   
1961  21906   highlightWomen             NaN     NaN     NaN      webapp.io   
1962  22856         isHiring             NaN     NaN     NaN       weweb.io   
1963  25977   highlightWomen             NaN     NaN     NaN        yhangry   
1964  29532  highlightLatinx             NaN     NaN     NaN    Ångström AI

/var/folders/cv/vszp_60n2yx3c1fttzx00nb40000gn/T/ipykernel_6146/401189302.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby('name').apply(choose_row).reset_index(drop=True)


In [58]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import pandas as pd

#Take the TFA and gets the number value
def getTotalFundingAmount(str):
    arr = str.split(' ')
    pattern = r'^\$\d+(\.\d+)?[a-zA-Z]$'
    for elem in arr:
        if(re.fullmatch(pattern, elem)):
            return elem[1:len(elem)-1]

#get number from a string
def getNum(str):
    match = re.search(r'\d+(\.\d+)?', str)
    if match:
        return match.group(0) 
    return None

# Set up Chrome options
options = Options()
options.add_experimental_option("detach", True)

# Initialize Web Driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

columns = ['Company', 'TFA', 'Funding Rounds', 'Investors', 'Lead Investors']
df = pd.DataFrame(columns=columns)

companies = ['54gene', 'weweb'] #For testing

# Navigate to the URL
for company in merged_df['company_slug']: #if testing, change merged_df['company_slug] to companies array
    driver.get(f"https://www.crunchbase.com/organization/{company}/company_financials")

    try:
        TFA=None
        funding_rounds=None
        l_investors=None
        investors=None
        # Locate the "Funding Rounds" label and its number
        right_divs = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'one-of-many-section') and contains(@class, 'ng-star-inserted')]"))
        )
        print(len(right_divs))
        first_div = right_divs[0]
        right_div = right_divs[1]
        new_text = right_div.text.replace('\n', ' ')
        print(new_text.split(' '))
        print(f'TFA: {getTotalFundingAmount(new_text)}')
        TFA = getTotalFundingAmount(new_text)
        print('')
        boxes = first_div.find_elements(By.XPATH, ".//div[@class='spacer ng-star-inserted']")
        for box in boxes:
            print(box.text)
            txt = box.text
            if "Funding Rounds" in txt:
                funding_rounds = getNum(txt)
            elif "Lead Investors" in txt:
                l_investors = getNum(txt)
            elif "Investors" in txt:
                investors = getNum(txt)
        new_row = pd.DataFrame({'Company': [company], 'TFA': [TFA], 'Funding Rounds': [funding_rounds], 'Lead Investors': [l_investors], 'Investors': [investors]})
        df = pd.concat([df, new_row], ignore_index=True)

    except Exception as e:
        print(f"An error occurred: {e}")

driver.quit()


An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.90)
Stacktrace:
0   chromedriver                        0x0000000100bb0d18 chromedriver + 4996376
1   chromedriver                        0x0000000100ba85da chromedriver + 4961754
2   chromedriver                        0x000000010074bd10 chromedriver + 388368
3   chromedriver                        0x000000010072344f chromedriver + 222287
4   chromedriver                        0x00000001007c49fd chromedriver + 883197
5   chromedriver                        0x00000001007da3f9 chromedriver + 971769
6   chromedriver                        0x00000001007bc753 chromedriver + 849747
7   chromedriver                        0x000000010078b635 chromedriver + 648757
8   chromedriver                        0x000000010078be5e chromedriver + 650846
9   chromedriver                        0x0000000100b77000 chromedriver + 4759552
10  chromedriver        

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.90)
Stacktrace:
0   chromedriver                        0x0000000100bb0d18 chromedriver + 4996376
1   chromedriver                        0x0000000100ba85da chromedriver + 4961754
2   chromedriver                        0x000000010074bd10 chromedriver + 388368
3   chromedriver                        0x000000010072344f chromedriver + 222287
4   chromedriver                        0x00000001007c49fd chromedriver + 883197
5   chromedriver                        0x00000001007da3f9 chromedriver + 971769
6   chromedriver                        0x00000001007bc753 chromedriver + 849747
7   chromedriver                        0x000000010078b635 chromedriver + 648757
8   chromedriver                        0x000000010078be5e chromedriver + 650846
9   chromedriver                        0x0000000100b77000 chromedriver + 4759552
10  chromedriver    

In [57]:
df.head()
for company in merged_df['company_slug']:
    print(company)

ai-2
10-by-10
1910-genetics
222
54gene
64x-bio
8vdx
99minutos
acx
alt-tex
aoa-dx-inc
aon3d
abatable
abstra
accord
actipulse-neuroscience
activeloop
aden
adla
adni
adra
adri-ai
advantage-club
adventris-pharmaceuticals
adventurous-co
aella
aer
aesculatech
aether-energy
affil-ai
affinity
afriex
afrostream
agape
agave
agency
agendapro
agilemd
ahazou
aiprise
airbnb
airfordable
airfront
ajaib
akiflow
akudo
alaan
alana
albiware-inc
aldara
alemhealth
aleph
aleph-solutions-alephee
alga-biosciences
algotest
algouniversity
algolia
alguna
alima
alinea
alixia
allsome-fulfillment
alloy-automation
alltruists
allure-systems
almond
alpaca
alpha-vantage
perch-credit
alude
amplitude
anakin
ananya-health
anaphero
ancana
anchor
andson-biotech
andy-ai
angle-health
anima
anima-app
anja-health
ankr-health
anniecannons
apartio
apero-health
apollo
apollo-io
apprecio
apprentice-health
apptimize
apriora
aptdeco
apto-payments
apurata
aquarium-learning
aragorn-ai
archform
arcus
ardis-ai
argovox
argument-computer-co